In [8]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
import os
import time
import sys

from liblibra_core import *
import libra_py.models.Holstein as Holstein
import util.libutil as comn
from libra_py import data_outs
import libra_py.data_savers as data_savers

import libra_py.dynamics.qtag.qtag_basis as qtag_basis
import libra_py.dynamics.qtag.qtag_calc as qtag_calc
import libra_py.dynamics.qtag.qtag_checks as qtag_checks
import libra_py.dynamics.qtag.qtag_ham as qtag_ham
import libra_py.dynamics.qtag.qtag_init as qtag_init
import libra_py.dynamics.qtag.qtag_mom as qtag_mom
import libra_py.dynamics.qtag.qtag_prop as qtag_prop
import libra_py.dynamics.qtag.qtag_save as save

In [9]:
class tmp:
    pass

#Holstein5 from libra_py/models/Hostein.py
def holstein_mimic(q, params, full_id):
    critical_params = ["E_n", "x_n", "k_n" ]
    default_params = { "V": [ [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001],
                              [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001] ],
                       "alpha": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                  [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                       "x_nm": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                 [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                     }
    comn.check_input(params, default_params, critical_params)

    E_n = params["E_n"]
    x_n = params["x_n"]
    k_n = params["k_n"]
    V = params["V"]
    alpha = params["alpha"]
    x_nm = params["x_nm"]
    
    n = len(E_n)

    Hdia = CMATRIX(n,n)
    Sdia = CMATRIX(n,n)
    d1ham_dia = CMATRIXList();  d1ham_dia.append( CMATRIX(n,n) )
    dc1_dia = CMATRIXList();  dc1_dia.append( CMATRIX(n,n) )

    #indx = 0
    Id = Cpp2Py(full_id)
    indx = Id[-1]

    x = q.col(indx).get(0)

    Sdia.identity()

    for i in range(n):
        Hdia.set(i,i,  (E_n[i] + 0.5*k_n[i]*(x - x_n[i])**2) * (1.0+0.0j) )

    for i in range(n):
        for j in range(n):
            if i!=j:
                Hdia.set(i,j,  V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    for k in [0]:
        #  d Hdia / dR_0
        for i in range(n):
            d1ham_dia[k].set(i,i, (k_n[i] * (x - x_n[i]))*(1.0+0.0j) )

    for k in [0]:
        for i in range(n):
            for j in range(n):
                if i!=j:
                    d1ham_dia[k].set(i,j,  -2.0*alpha[i][j] * (x-x_nm[i][j]) * V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    obj = tmp()
    obj.ham_dia = Hdia
    obj.ovlp_dia = Sdia
    obj.d1ham_dia = d1ham_dia
    obj.dc1_dia = dc1_dia

    return obj

In [10]:
def compute_model(q, model_params, full_id):

    model = model_params["model"]
    res = None
    
    if model==1:
        #res = Holstein.Holstein2(q, model_params, full_id)
        res = holstein_mimic(q, model_params, full_id)
    else:
        print("Unrecognized model! Check for correct model import statement.")

    return res

In [11]:
def run_qtag(dyn_params, _compute_model, _model_params, prefix):

    default_params = {
        "hdf5_output_level":-1, "prefix":prefix, "use_compression":0, "compression_level":[0,0,0], 
        "mem_output_level":4, "txt2_output_level":0, "properties_to_save": [], "progress_frequency": 1
    }
    critical_params = []
    
    comn.check_input(dyn_params, default_params, critical_params)
#    qtag_checks.user_input(dyn_params, model_params)
    #Define function objects to be used in various parts of the calculation...
    #initialize, basis_props, vapprox, mom_calc, propagate = qtag_checks.assign_fobj(qtag_params)

    basis_props = qtag_checks.set_basis_updates(dyn_params)
    
    #Rename variables locally for convenience...
    ndof = dyn_params["ndof"]
    nstates = dyn_params["nstates"]
    active_state = dyn_params["active_state"]
    nsteps = dyn_params["nsteps"]
    dt = dyn_params["dt"]
    mass = dyn_params["mass"]
    iM = dyn_params["iM"]
    #iM = MATRIX(1, 1);  iM.set(0,0, 1.0);

    #Initialize the basis parameters {q,p,a,s} and create initial b-matrix...
    ntraj, qpas = qtag_init.initialize(dyn_params)

    #Create initial projection vector b...
    bt = qtag_init.coeffs(dyn_params, qpas, active_state)

    #Initialize savers...
    dyn_params['prefix']  = prefix
    properties_to_save = dyn_params['properties_to_save']
    _savers = save.init_qtag_savers(dyn_params, _model_params, nsteps, ntraj, ndof, nstates)
    
    #Start simulation and walltime variables...
    walltime_start = time.time()
    t=0.0

    
    ham = nHamiltonian(nstates, nstates, ndof)
    ham.add_new_children(nstates, nstates, ndof, ntraj)    
    ham.init_all(2,1)
    _model_params.update({"timestep":0})
    
    ovlp = CMATRIX(ntraj, ntraj)
    hmat = CMATRIX(ntraj, ntraj)
    
    coeff = CMATRIX(nstates, ntraj)
    
    
    for step in range(nsteps):
        

#=======================OLD PYTHON WAY FOR COMPUTING HAMILTONIAN==========================================
        #Calculate system overlap on each state, placed into a super-matrix ovlp...
        #ovlp = qtag_calc.super_overlap(ndof,ntraj,nstates,qpas)

        #Calculate system Hamiltonian for all states and couplings, placed into a super-matrix hmat...
        #hmat = qtag_ham.super_hamiltonian(ndof,ntraj,nstates,mass,qpas, \
        #                                  dyn_params["qtag_pot_approx_method"], \
        #                                  _compute_model,_model_params)

        #Compute the new coefficient vector c_new...
#        ct_new = qtag_ham.basis_diag(ntraj,dt,hmat,ovlp,bt)
#=========================================================================================================        
        #print(qpas[4])
    
        qtag_hamiltonian_and_overlap(qpas[0], qpas[1], qpas[2], qpas[3], coeff, 
                                     Py2Cpp_int(qpas[4]), iM, ham, 
                                     _compute_model, _model_params, dyn_params,
                                     ovlp, hmat)
        
        #Compute the new coefficient vector c_new...
        ct_new = qtag_ham.basis_diag(ntraj,dt,hmat,ovlp,bt)
            
        etot = qtag_calc.energy(ct_new,hmat)
        pops = qtag_calc.norm(qpas[4],ct_new,ovlp,
                              [state for state in range(nstates)])

        print(etot)
        #Update the basis parameters according to the new wavefunction (ct_new)...
        qpas, bt = qtag_prop.propagate(dyn_params, basis_props,qpas,ct_new,pops)
        
        save.save_qtag_data(_savers, dyn_params, step, etot, 0, pops, 
                            ct_new, qpas[0], qpas[1], qpas[2], qpas[3])        
        if _savers["txt2_saver"]!=None:
            _savers["txt2_saver"].save_data_txt( F"{prefix}", properties_to_save, "a", 0)
            
    print(pops)
    walltime_end = time.time()
    print("Total wall time: ",walltime_end-walltime_start)


In [12]:
iM = MATRIX(1, 1);  iM.set(0,0, 1.0);

#Parameters for general simulation
dyn_params = {
    "ndof" : 1, "nstates" : 2, "active_state" : 0,
    "dt":10., "nsteps":20, "mass":[1.0], "iM":iM,
    "hdf5_output_level" : 0, "txt2_output_level": 3, 
    "properties_to_save" : ["time","Etot","pops","q"],
    "init_placement":0, "grid_dims" : [5], "alp_scl":[6.0], "rho_cut":1e-12,
    "basis_qtype" : 1, "basis_ptype" : 1, "basis_atype" : 1, "basis_stype" : 0,
    "wfc_q0" : [-2.0], "wfc_p0" : [0.0], "wfc_a0" : [1.0], "wfc_s0" : [0.0],
    "prop_method" : 1, "decpl_den" : 0.3,
    "mom_calc_type" : 1, "linfit_beta" : 1e-3,
    "qtag_pot_approx_method":0
}

#Potential parameters for Holstein5
model_params = {
    "model":1,
    "E_n":[0.0,15.8114],
    "x_n":[0.0,1.0],
    "k_n":[10.0,10.0],
    "V":[[0.0,1.0],[1.0,0.0]],
    "alpha":[[0.0,1.5811],[1.5811,0.0]],
    "x_nm":[[0.0,2.0],[2.0,0.0]]
}

run_qtag(dyn_params, compute_model, model_params, "qtag")

15.050030925787372
 step= 0
15.050030925787382
15.050030925787397
15.05003092578742
15.050030925787441
15.050030925787452
15.050030925787473
15.05003092578749
15.050030925787501
15.050030925787519
15.050030925787526
15.050030925787553
15.050030925787569
15.05003092578759
15.050030925787606
15.050030925787624
15.050030925787647
15.050030925787663
15.050030925787684
15.050030925787706
[0.8743601549642422, 8.70075639668817e-09]
Total wall time:  0.24143028259277344
